#### URLS

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>url</th>
      <th>title</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td><a href="https://player.vimeo.com/video/656704385?h=abf81d298d">https://player.vimeo.com/video/656704385?h=abf81d298d</a></td>
      <td>4</td>
    </tr>
    <tr>
      <th>1</th>
      <td><a href="https://player.vimeo.com/video/656703362?h=bae256298f">https://player.vimeo.com/video/656703362?h=bae256298f</a></td>
      <td>8</td>
    </tr>
    <tr>
      <th>2</th>
      <td><a href="https://player.vimeo.com/video/656701336?h=c3a3e9bc16">https://player.vimeo.com/video/656701336?h=c3a3e9bc16</a></td>
      <td>12</td>
    </tr>
    <tr>
      <th>3</th>
      <td><a href="https://player.vimeo.com/video/656697884?h=95081c955c">https://player.vimeo.com/video/656697884?h=95081c955c</a></td>
      <td>18</td>
    </tr>
    <tr>
      <th>4</th>
      <td><a href="https://player.vimeo.com/video/656697539?h=9e0a4673f0">https://player.vimeo.com/video/656697539?h=9e0a4673f0</a></td>
      <td>23</td>
    </tr>
    <tr>
      <th>5</th>
      <td><a href="https://player.vimeo.com/video/656697049?h=649a69e5a2">https://player.vimeo.com/video/656697049?h=649a69e5a2</a></td>
      <td>27</td>
    </tr>
    <tr>
      <th>6</th>
      <td><a href="https://player.vimeo.com/video/656696370?h=a809e66bb8">https://player.vimeo.com/video/656696370?h=a809e66bb8</a></td>
      <td>31</td>
    </tr>
    <tr>
      <th>7</th>
      <td><a href="https://player.vimeo.com/video/656696079?h=80b4e6ce8e">https://player.vimeo.com/video/656696079?h=80b4e6ce8e</a></td>
      <td>36</td>
    </tr>
    <tr>
      <th>8</th>
      <td><a href="https://player.vimeo.com/video/656749759?h=095ad450ac">https://player.vimeo.com/video/656749759?h=095ad450ac</a></td>
      <td>40</td>
    </tr>
    <tr>
      <th>9</th>
      <td><a href="https://player.vimeo.com/video/656688282?h=84ef8e90b3">https://player.vimeo.com/video/656688282?h=84ef8e90b3</a></td>
      <td>43</td>
    </tr>
    <tr>
      <th>10</th>
      <td><a href="https://player.vimeo.com/video/656688065?h=c391dae2e6">https://player.vimeo.com/video/656688065?h=c391dae2e6</a></td>
      <td>48</td>
    </tr>
    <tr>
      <th>11</th>
      <td><a href="https://player.vimeo.com/video/656687537?h=67df9f3bd7">https://player.vimeo.com/video/656687537?h=67df9f3bd7</a></td>
      <td>55</td>
    </tr>
    <tr>
      <th>12</th>
      <td><a href="https://player.vimeo.com/video/656687259?h=684b40ef32">https://player.vimeo.com/video/656687259?h=684b40ef32</a></td>
      <td>59</td>
    </tr>
    <tr>
      <th>13</th>
      <td><a href="https://player.vimeo.com/video/656686948?h=2dbbdccfa4">https://player.vimeo.com/video/656686948?h=2dbbdccfa4</a></td>
      <td>63</td>
    </tr>
    <tr>
      <th>14</th>
      <td><a href="https://player.vimeo.com/video/656686010?h=214406a99f">https://player.vimeo.com/video/656686010?h=214406a99f</a></td>
      <td>67</td>
    </tr>
    <tr>
      <th>15</th>
      <td><a href="https://player.vimeo.com/video/656685822?h=6b6bce7f3c">https://player.vimeo.com/video/656685822?h=6b6bce7f3c</a></td>
      <td>73</td>
    </tr>
    <tr>
      <th>16</th>
      <td><a href="https://player.vimeo.com/video/656685645?h=444e6e49e7">https://player.vimeo.com/video/656685645?h=444e6e49e7</a></td>
      <td>78</td>
    </tr>
    <tr>
      <th>17</th>
      <td><a href="https://player.vimeo.com/video/656685380?h=3b79fe2cdb">https://player.vimeo.com/video/656685380?h=3b79fe2cdb</a></td>
      <td>84</td>
    </tr>
    <tr>
      <th>18</th>
      <td><a href="https://player.vimeo.com/video/656685229?h=b668f12bc1">https://player.vimeo.com/video/656685229?h=b668f12bc1</a></td>
      <td>89</td>
    </tr>
    <tr>
      <th>19</th>
      <td><a href="https://player.vimeo.com/video/656684478?h=87d29a2ba6">https://player.vimeo.com/video/656684478?h=87d29a2ba6</a></td>
      <td>98</td>
    </tr>
    <tr>
      <th>20</th>
      <td><a href="https://player.vimeo.com/video/656684245?h=f96cf91211">https://player.vimeo.com/video/656684245?h=f96cf91211</a></td>
      <td>102</td>
    </tr>
    <tr>
      <th>21</th>
      <td><a href="https://player.vimeo.com/video/656684037?h=f30c2b4dfc">https://player.vimeo.com/video/656684037?h=f30c2b4dfc</a></td>
      <td>106</td>
    </tr>
    <tr>
      <th>22</th>
      <td><a href="https://player.vimeo.com/video/656683862?h=886904448d">https://player.vimeo.com/video/656683862?h=886904448d</a></td>
      <td>110</td>
    </tr>
  </tbody>
</table>

#### Predicting House with size